In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [73]:
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import pandas as pd
import numpy as np
from surprise import accuracy, Dataset, SVD,SVDpp, NMF
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split as train_test_split_surprise
from tqdm.notebook import tqdm

import numpy as np
from sklearn.metrics import ndcg_score
import scipy.stats as ss

In [104]:
import sys
from os.path import dirname
sys.path.append(dirname("../../"))

from src.utilities import *
from src.reccomendation_util import *

In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Model and Raw Data

In [78]:
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
print(raw_interactions.shape)

(1132367, 5)


In [71]:
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata.shape

(231637, 13)

In [38]:
### Load Models
model1 = load_pickle("../../models/reccomender_model1_svd.pkl")
model2 = load_pickle("../../models/reccomender_model2_nmf.pkl")

In [55]:
model1.predict(uid=1533,iid=44459)

Prediction(uid=1533, iid=44459, r_ui=None, est=4.979205256134577, details={'was_impossible': False})

In [54]:
model1.test([[1533,57549,None],\
            [1533,44459,None],\
            [1533,31048,None],\
            [1533,10554,None]])

[Prediction(uid=1533, iid=57549, r_ui=None, est=4.984623436267778, details={'was_impossible': False}),
 Prediction(uid=1533, iid=44459, r_ui=None, est=4.979205256134577, details={'was_impossible': False}),
 Prediction(uid=1533, iid=31048, r_ui=None, est=4.909425973113856, details={'was_impossible': False}),
 Prediction(uid=1533, iid=10554, r_ui=None, est=4.904533254551073, details={'was_impossible': False})]

In [69]:
model2.test([[1533,57549,None],\
            [1533,44459,None],\
            [1533,31048,None],\
            [1533,10554,None]])

[Prediction(uid=1533, iid=57549, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=44459, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=31048, r_ui=None, est=5, details={'was_impossible': False}),
 Prediction(uid=1533, iid=10554, r_ui=None, est=5, details={'was_impossible': False})]

### Meta

In [88]:
recipe_metadata['nutrition_list'] = recipe_metadata['nutrition'].apply(lambda x: ast.literal_eval(x))
recipe_metadata[['calories','fat_dv',"sugar_dv",\
           "sodium_dv","protein_dv","sat_fat", "carbs_dv"]\
      ] = pd.DataFrame(recipe_metadata.nutrition_list.tolist(), index= recipe_metadata.index)

In [92]:
example_ingredients = ['apple juice', 'raw honey', 'whole cloves',\
                        'everclear alcohol', 'ground cinnamon', 'cheesecloth']

In [90]:
recipe_metadata['ingredients_list'] = recipe_metadata['ingredients'].apply(lambda x: ast.literal_eval(x))

In [100]:
def match_ingredients(x,to_match, threshold = 0.25):
    x = set(x)
    to_match = set(to_match)
    match_rate = len(x.intersection(to_match))/len(x)
    return match_rate>=threshold

In [103]:
recipe_metadata[recipe_metadata["ingredients_list"].apply(lambda x: match_ingredients(x,example_ingredients))]

,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,...,n_ingredients,nutrition_list,calories,fat_dv,sugar_dv,sodium_dv,protein_dv,sat_fat,carbs_dv,ingredients_list
317,317,apple pie,124853,43200,213699,2005-06-06,"['time-to-make', 'course', 'main-ingredient', ...","[285.8, 0.0, 281.0, 0.0, 0.0, 0.0, 24.0]",2,['mix ingredence together except everclear and...,...,6,"[285.8, 0.0, 281.0, 0.0, 0.0, 0.0, 24.0]",285.8,0.0,281.0,0.0,0.0,0.0,24.0,"[apple juice, raw honey, whole cloves, evercle..."
820,820,10 minute applesauce alton brown,121344,20,66322,2005-05-10,"['30-minutes-or-less', 'time-to-make', 'course...","[270.1, 9.0, 189.0, 2.0, 1.0, 18.0, 19.0]",5,"['in a sealable microwave-safe container , com...",...,7,"[270.1, 9.0, 189.0, 2.0, 1.0, 18.0, 19.0]",270.1,9.0,189.0,2.0,1.0,18.0,19.0,"[golden delicious apples, fuji apples, apple j..."
2282,2282,absolut vince vega,261358,5,327115,2007-10-25,"['15-minutes-or-less', 'time-to-make', 'course...","[65.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2,"['build over ice in a highball glass', 'decora...",...,2,"[65.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",65.5,0.0,0.0,0.0,0.0,0.0,0.0,"[absolut citron vodka, apple juice]"
2634,2634,advieh,253004,5,292560,2007-09-14,"['15-minutes-or-less', 'time-to-make', 'course...","[98.2, 3.0, 1.0, 0.0, 5.0, 1.0, 7.0]",5,['whizz the rose petals in the spice grinder u...,...,4,"[98.2, 3.0, 1.0, 0.0, 5.0, 1.0, 7.0]",98.2,3.0,1.0,0.0,5.0,1.0,7.0,"[dried rose petals, ground cinnamon, ground ca..."
3555,3555,almond banana shake,340071,4,599450,2008-11-30,"['15-minutes-or-less', 'time-to-make', 'course...","[111.5, 0.0, 57.0, 0.0, 2.0, 0.0, 9.0]",2,"['add banana slices , almond milk , and cinnam...",...,3,"[111.5, 0.0, 57.0, 0.0, 2.0, 0.0, 9.0]",111.5,0.0,57.0,0.0,2.0,0.0,9.0,"[banana, almond milk, ground cinnamon]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227737,227737,winter spice how nice warm breakfast drink,275801,4,37305,2008-01-03,"['15-minutes-or-less', 'time-to-make', 'course...","[107.0, 1.0, 0.0, 6.0, 19.0, 2.0, 5.0]",2,"['place milk , carnation instant breakfast , g...",...,4,"[107.0, 1.0, 0.0, 6.0, 19.0, 2.0, 5.0]",107.0,1.0,0.0,6.0,19.0,2.0,5.0,"[skim milk, instant breakfast drink mix, groun..."
227739,227739,winter apple jelly,484618,35,2344388,2012-07-30,"['60-minutes-or-less', 'time-to-make', 'course...","[698.4, 0.0, 682.0, 0.0, 0.0, 0.0, 60.0]",12,"['in a dutch oven , combine the apple juice , ...",...,6,"[698.4, 0.0, 682.0, 0.0, 0.0, 0.0, 60.0]",698.4,0.0,682.0,0.0,0.0,0.0,60.0,"[apple juice, powdered fruit pectin, lemon jui..."
227932,227932,witches brew,37830,5,37636,2002-08-20,"['15-minutes-or-less', 'time-to-make', 'course...","[108.8, 0.0, 104.0, 0.0, 0.0, 0.0, 9.0]",4,"['combine all ingredients and chill', 'pour in...",...,4,"[108.8, 0.0, 104.0, 0.0, 0.0, 0.0, 9.0]",108.8,0.0,104.0,0.0,0.0,0.0,9.0,"[frozen lemonade concentrate, apple juice, gin..."
229398,229398,yogurt cheese,56555,5760,10404,2003-03-18,"['weeknight', 'time-to-make', 'course', 'main-...","[254.2, 1.0, 139.0, 14.0, 52.0, 2.0, 11.0]",7,['set a strainer lined with 2 layers of cheese...,...,2,"[254.2, 1.0, 139.0, 14.0, 52.0, 2.0, 11.0]",254.2,1.0,139.0,14.0,52.0,2.0,11.0,"[nonfat yogurt, cheesecloth]"


### Reccomendation Testting

In [87]:
gen_recipe_list(raw_interactions,1533)

array([ 40893,  44394,  85009, ..., 257796,  72730, 386618])

In [80]:
gen_recipe_list(raw_interactions,1533)

array([ 40893,  44394,  85009, ..., 257796,  72730, 386618])

In [82]:
get_reccomendation_ids(model1,1533,gen_recipe_list(raw_interactions,1533),20)

[123665,
 27509,
 379639,
 49461,
 265683,
 235347,
 119804,
 515167,
 138647,
 319562,
 97606,
 92649,
 524863,
 192308,
 152898,
 61195,
 72119,
 24044,
 16603,
 134316]

In [83]:
rec_ids = get_reccomendation_ids(model1,1533,gen_recipe_list(raw_interactions,1533),20)

In [86]:
recipe_meta_map(recipe_metadata,rec_ids)

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation/../../src/reccomendation_util.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return df_subset


,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,recipe_id
165774,165774,preserved lemons,16603,7205,21752,2002-01-02,"['weeknight', 'time-to-make', 'course', 'main-...","[16.8, 1.0, 2.0, 65.0, 0.0, 0.0, 0.0]",8,"['blanch 6 lemons in boiling water 5 minutes',...",this is an adaptation of paula wolfert's quick...,"['lemons', 'coarse salt', 'olive oil']",3,16603
140186,140186,mrs schreiner s split pea soup,24044,140,8688,2002-03-31,"['weeknight', 'time-to-make', 'course', 'main-...","[306.3, 16.0, 22.0, 21.0, 33.0, 17.0, 12.0]",9,['in a large heavy kettle or stock pot cook ba...,my family loves split-pea soup and this is the...,"['bacon', 'onion', 'carrots', 'dried split pea...",8,24044
125585,125585,lost bread french toast,27509,25,36998,2002-05-07,"['30-minutes-or-less', 'time-to-make', 'course...","[234.3, 7.0, 161.0, 2.0, 10.0, 10.0, 14.0]",10,['mix both dry ingredients until thoroughly co...,"when i was a little girl, my mama always made ...","['sugar', 'cinnamon', 'eggs', 'milk', 'vanilla...",7,27509
15483,15483,baked vegetarian chimichangas warm or cold,49461,25,39547,2002-12-22,"['30-minutes-or-less', 'time-to-make', 'course...","[250.1, 15.0, 15.0, 24.0, 23.0, 26.0, 9.0]",9,"['saut the chopped onion , garlic , and mushro...","these are a deliciously healthy, filling meal,...","['onion', 'garlic cloves', 'mushroom', 'olive ...",17,49461
139891,139891,moxie s crystallized ginger cookies,61195,85,55380,2003-05-05,"['weeknight', 'time-to-make', 'course', 'main-...","[90.4, 6.0, 27.0, 4.0, 1.0, 11.0, 4.0]",9,"['preheat oven to 375', 'cream butter& sesame ...",this was originally posted by my cyber-friend ...,"['unsalted butter', 'sesame oil', 'dark molass...",12,61195
185902,185902,shrimp parmesan,72119,55,58104,2003-10-01,"['60-minutes-or-less', 'time-to-make', 'course...","[860.7, 42.0, 40.0, 92.0, 110.0, 68.0, 31.0]",17,['open the shrimp up like a book and remove th...,my husband favorite. a one dish meal. you can ...,"['large shrimp', 'flour', 'old bay seasoning',...",12,72119
26220,26220,blueberry pancakes milk free egg free,92649,15,40135,2004-06-04,"['lactose', '15-minutes-or-less', 'time-to-mak...","[162.3, 8.0, 30.0, 12.0, 6.0, 6.0, 8.0]",15,"['sift flour , sugar , baking powder , and sal...",a seriously delicious recipe. we pick our wild...,"['white flour', 'white sugar', 'baking powder'...",8,92649
59201,59201,country sausage gravy,97606,35,134663,2004-08-12,"['60-minutes-or-less', 'time-to-make', 'course...","[378.2, 46.0, 0.0, 32.0, 32.0, 65.0, 3.0]",8,['brown the pork sausage in a large skillet ov...,"growing up, we ate this gravy over slices of w...","['pork sausage', 'butter', 'all-purpose flour'...",6,97606
117962,117962,kittencal s banana cinnamon snack cake or muff...,119804,30,89831,2005-04-30,"['30-minutes-or-less', 'time-to-make', 'course...","[183.1, 5.0, 59.0, 15.0, 9.0, 3.0, 10.0]",11,"['set oven to 350 degrees f', 'grease a 8 or 9...",a wonderful lower-fat cake/muffin recipe that'...,"['all-purpose flour', 'sugar', 'baking powder'...",11,119804
32135,32135,buttermilk buckwheat pancakes gluten free,123665,25,157102,2005-05-26,"['30-minutes-or-less', 'time-to-make', 'course...","[195.9, 8.0, 41.0, 27.0, 14.0, 13.0, 10.0]",6,"['combine dry ingredients in large bowl', 'whi...",i’m posting this mainly for those of you on za...,"['buckwheat flour', 'brown sugar', 'baking pow...",11,123665


### Test final function

In [108]:
example_ingredients

['apple juice',
 'raw honey',
 'whole cloves',
 'everclear alcohol',
 'ground cinnamon',
 'cheesecloth']

In [ ]:
#Read data
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata['nutrition_list'] = recipe_metadata['nutrition'].apply(lambda x: ast.literal_eval(x))
recipe_metadata['ingredients_list'] = recipe_metadata['ingredients'].apply(lambda x: ast.literal_eval(x))
recipe_metadata[['calories', 'fat_dv', "sugar_dv", \
                     "sodium_dv", "protein_dv", "sat_fat", "carbs_dv"] \
               ] = pd.DataFrame(recipe_metadata.nutrition_list.tolist(), index=recipe_metadata.index)

In [114]:
###Basic
ad_final_reccom(1533,example_ingredients)

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation/../../src/reccomendation_util.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return df_subset


,name,id,rating_pred,minutes,ingredients_list,nutrition_list
168917,quick elephant ears,9085,5.000000,10,"[sugar, ground cinnamon, oil, flour tortillas]","[210.9, 3.0, 122.0, 7.0, 5.0, 2.0, 15.0]"
188651,slow cooked cinnamon applesauce,16587,4.940929,190,"[granny smith apples, brown sugar, fresh lemon...","[156.2, 0.0, 136.0, 0.0, 1.0, 0.0, 13.0]"
50882,chocolate malt coffee creamer,46632,4.934600,0,"[powdered cocoa mix, non-dairy powdered coffee...","[189.9, 3.0, 132.0, 8.0, 6.0, 8.0, 12.0]"
35820,caramelized bacon,51890,4.933929,20,"[brown sugar, ground cinnamon, bacon]","[660.9, 78.0, 141.0, 40.0, 26.0, 85.0, 12.0]"
143099,nelda s cinnamon pecans,107559,4.929066,15,"[pecan halves, granulated sugar, ground cinnam...","[880.8, 109.0, 215.0, 0.0, 18.0, 30.0, 21.0]"
194904,spiced pear jam,116767,4.928001,180,"[pears, sugar, ground cinnamon, ground cloves]","[770.3, 0.0, 739.0, 0.0, 2.0, 0.0, 66.0]"
220653,vanilla poached pears,133911,4.913782,70,"[pears, apple juice, vanilla beans, ground cin...","[66.7, 0.0, 45.0, 0.0, 0.0, 0.0, 5.0]"
131369,marshmallow fruit dip,150771,4.907214,5,"[marshmallow creme, cream cheese, vanilla extr...","[471.0, 24.0, 180.0, 8.0, 8.0, 50.0, 25.0]"
143396,new orleans caf au lait,151134,4.882978,5,"[coffee and chicory blend, milk, ground cinnamon]","[79.3, 6.0, 0.0, 2.0, 8.0, 13.0, 1.0]"
4452,amaretto coffee creamer,197284,4.872818,5,"[non-dairy coffee creamer, almond extract, gro...","[153.2, 6.0, 109.0, 1.0, 0.0, 4.0, 9.0]"


In [116]:
#Read data
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata['nutrition_list'] = recipe_metadata['nutrition'].apply(lambda x: ast.literal_eval(x))
recipe_metadata['ingredients_list'] = recipe_metadata['ingredients'].apply(lambda x: ast.literal_eval(x))
recipe_metadata[['calories', 'fat_dv', "sugar_dv", \
                     "sodium_dv", "protein_dv", "sat_fat", "carbs_dv"]\
               ] = pd.DataFrame(recipe_metadata.nutrition_list.tolist(), index=recipe_metadata.index)

In [117]:
protein_mask = recipe_metadata["protein_dv"]>20.0

In [118]:
###Basic
ad_final_reccom(1533,example_ingredients,raw_interactions, recipe_metadata)

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation/../../src/reccomendation_util.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return df_subset


,name,id,rating_pred,minutes,ingredients_list,nutrition_list
168917,quick elephant ears,9085,5.000000,10,"[sugar, ground cinnamon, oil, flour tortillas]","[210.9, 3.0, 122.0, 7.0, 5.0, 2.0, 15.0]"
188651,slow cooked cinnamon applesauce,16587,4.940929,190,"[granny smith apples, brown sugar, fresh lemon...","[156.2, 0.0, 136.0, 0.0, 1.0, 0.0, 13.0]"
50882,chocolate malt coffee creamer,46632,4.934600,0,"[powdered cocoa mix, non-dairy powdered coffee...","[189.9, 3.0, 132.0, 8.0, 6.0, 8.0, 12.0]"
35820,caramelized bacon,51890,4.933929,20,"[brown sugar, ground cinnamon, bacon]","[660.9, 78.0, 141.0, 40.0, 26.0, 85.0, 12.0]"
143099,nelda s cinnamon pecans,107559,4.929066,15,"[pecan halves, granulated sugar, ground cinnam...","[880.8, 109.0, 215.0, 0.0, 18.0, 30.0, 21.0]"
194904,spiced pear jam,116767,4.928001,180,"[pears, sugar, ground cinnamon, ground cloves]","[770.3, 0.0, 739.0, 0.0, 2.0, 0.0, 66.0]"
220653,vanilla poached pears,133911,4.913782,70,"[pears, apple juice, vanilla beans, ground cin...","[66.7, 0.0, 45.0, 0.0, 0.0, 0.0, 5.0]"
131369,marshmallow fruit dip,150771,4.907214,5,"[marshmallow creme, cream cheese, vanilla extr...","[471.0, 24.0, 180.0, 8.0, 8.0, 50.0, 25.0]"
143396,new orleans caf au lait,151134,4.882978,5,"[coffee and chicory blend, milk, ground cinnamon]","[79.3, 6.0, 0.0, 2.0, 8.0, 13.0, 1.0]"
4452,amaretto coffee creamer,197284,4.872818,5,"[non-dairy coffee creamer, almond extract, gro...","[153.2, 6.0, 109.0, 1.0, 0.0, 4.0, 9.0]"


In [120]:
###With Protein mask
ad_final_reccom(1533,example_ingredients,raw_interactions, recipe_metadata,\
                extra_filters=[protein_mask])

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation/../../src/reccomendation_util.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return df_subset


,name,id,rating_pred,minutes,ingredients_list,nutrition_list
208865,tasty apple kielbasa,33253,4.933929,20,"[kielbasa, apple juice, apple jelly, maple syrup]","[409.3, 47.0, 56.0, 42.0, 27.0, 52.0, 6.0]"
54044,cinnamon sugar pull apart loaf,40346,4.844620,42,"[sugar, ground cinnamon, butter, refrigerated ...","[2297.5, 212.0, 510.0, 184.0, 44.0, 349.0, 84.0]"
35820,caramelized bacon,51890,4.834162,20,"[brown sugar, ground cinnamon, bacon]","[660.9, 78.0, 141.0, 40.0, 26.0, 85.0, 12.0]"
229398,yogurt cheese,56555,4.803867,5760,"[nonfat yogurt, cheesecloth]","[254.2, 1.0, 139.0, 14.0, 52.0, 2.0, 11.0]"
213366,thunderbird roast,59780,4.795691,485,"[chuck roast, dry onion soup mix, brown gravy ...","[400.6, 22.0, 37.0, 42.0, 98.0, 31.0, 6.0]"
130376,maple glazed sausages,70684,4.769402,35,"[sausage links, maple syrup, brown sugar, grou...","[440.0, 31.0, 198.0, 20.0, 22.0, 33.0, 18.0]"
99569,grilled pork tenderloin with pepper jelly glaze,78879,4.769036,120,"[apple juice, apple cider vinegar, pepper jell...","[752.1, 19.0, 335.0, 6.0, 93.0, 21.0, 37.0]"
53778,cinnamon latte,81869,4.756724,10,"[strong coffee, ground cinnamon, sugar, 1% low...","[158.9, 4.0, 90.0, 5.0, 21.0, 10.0, 7.0]"
206004,sweet glazed pork roast,83347,4.741153,100,"[pork tenderloin, preserves, apple juice]","[692.4, 28.0, 161.0, 8.0, 140.0, 31.0, 18.0]"
130263,maple syrup glazed ham jambon glac a l rable,134293,4.739035,360,"[ham, maple syrup, water, raisins, maple sugar...","[1199.2, 45.0, 400.0, 323.0, 230.0, 49.0, 38.0]"


In [121]:
###With Protein mask
ad_final_reccom(1533,example_ingredients,raw_interactions, recipe_metadata,\
                extra_filters=[protein_mask],threshold=0.4)

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation/../../src/reccomendation_util.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return df_subset


,name,id,rating_pred,minutes,ingredients_list,nutrition_list
229398,yogurt cheese,56555,4.803867,5760,"[nonfat yogurt, cheesecloth]","[254.2, 1.0, 139.0, 14.0, 52.0, 2.0, 11.0]"
91433,german quark,153126,4.688684,1800,"[buttermilk, cheesecloth]","[3090.5, 104.0, 1480.0, 338.0, 511.0, 211.0, 1..."
108585,hot apple oatmeal,237302,4.658352,15,"[oats, apple juice, unsweetened applesauce, gr...","[386.9, 8.0, 70.0, 0.0, 26.0, 4.0, 24.0]"
34180,california raisin peanut butter spread,255812,3.627838,20,"[raisins, apple juice, honey, ground cinnamon,...","[2040.0, 201.0, 545.0, 50.0, 136.0, 133.0, 63.0]"
